In [65]:
# Dependencies
# !pip install sqlalchemy 
# !pip install psycopg2-binary
# !pip install pandas
# !pip install openpyxl

In [66]:
from sqlalchemy import create_engine, text, inspect
from dotenv import load_dotenv
import pandas as pd
import os

In [67]:
# Establish DB connection

load_dotenv()    # loads .env from current directory by default
db_url = os.getenv("DATABASE_URL") # make sure to get the proper .env file from kristina !
engine = create_engine(db_url)

with engine.connect() as conn:
    result = conn.execute(text("SELECT NOW();"))
    print(result.fetchone())


(datetime.datetime(2025, 11, 23, 0, 26, 19, 419627, tzinfo=datetime.timezone.utc),)


In [68]:
# Verify connection by getting db table names

insp = inspect(engine)
tables = insp.get_table_names()

print(tables)

['custom_poi', 'poi_alias', 'spatial_ref_sys', 'osm2pgsql_properties', 'planet_osm_rels', 'planet_osm_nodes', 'planet_osm_ways', 'planet_osm_point', 'planet_osm_line', 'planet_osm_polygon', 'planet_osm_roads']


In [69]:
# To drop poi_alias table

# with engine.connect() as conn:
#     conn.execute(text("DROP TABLE IF EXISTS poi_alias CASCADE;"))
#     conn.commit()

In [70]:
# Create the custom_poi table

create_sql = """
CREATE TABLE IF NOT EXISTS poi_alias (
    alias_id SERIAL PRIMARY KEY,
    osm_table TEXT NOT NULL CHECK (osm_table IN ('point', 'polygon', 'line')),
    osm_object BIGINT NOT NULL,
    custom_poi INT DEFAULT NULL,
    alias TEXT NOT NULL,
    type TEXT NOT NULL CHECK (type IN ('abbr', 'alt')),
    UNIQUE (osm_table, alias)
);

"""

with engine.connect() as conn:
    conn.execute(text(create_sql))
    conn.commit()

print("poi_alias table created!")

poi_alias table created!


In [71]:
# Validation functions

# Check if osm object exists
def osm_object_exists(engine, osm_table, osm_id):
    table_map = {
        "point": "planet_osm_point",
        "polygon": "planet_osm_polygon",
        "line": "planet_osm_line"
    }
    table = table_map.get(osm_table)
    if not table:
        return False

    q = text(f"SELECT 1 FROM {table} WHERE osm_id = :oid LIMIT 1")

    # each call opens a clean connection
    with engine.connect() as conn:
        result = conn.execute(q, {"oid": osm_id}).fetchone()
        return bool(result)


# Check that custom_poi exists (poi_id is optional)
def custom_poi_exists(engine, poi_id):
    if poi_id is None or pd.isna(poi_id):
        return True  # optional relationship

    q = text("SELECT 1 FROM custom_poi WHERE poi_id = :pid LIMIT 1")

    with engine.connect() as conn:
        result = conn.execute(q, {"pid": poi_id}).fetchone()
        return bool(result)


In [72]:
df = pd.read_excel("poi_alias_list.xlsx")

existing_aliases = set()

with engine.begin() as conn:   # automatic commit
    for _, row in df.iterrows():
        osm_table = row['osm_table']
        osm_object = row['osm_object']
        custom_poi = row.get('custom_poi')
        alias = row['alias']
        alias_type = row['type']

        key = (osm_table, osm_object, alias, alias_type)

        # Skip duplicates from spreadsheet
        if key in existing_aliases:
            print(f"Skipping duplicate in CSV: {key}")
            continue

        # --- Validate OSM object ---
        if not osm_object_exists(engine, osm_table, osm_object):
            print(f"Skipping: osm_object {osm_object} not found in {osm_table}")
            continue

        # --- Validate custom_poi (only if provided) ---
        if not custom_poi_exists(engine, custom_poi):
            print(f"Skipping: custom_poi '{custom_poi}' not found")
            continue

        # --- Insert ---
        insert_sql = text("""
            INSERT INTO poi_alias (osm_table, osm_object, custom_poi, alias, type)
            VALUES (:osm_table, :osm_object, :custom_poi, :alias, :type)
            ON CONFLICT (osm_table, alias) DO NOTHING
            RETURNING alias_id
        """)

        conn.execute(insert_sql, {
            "osm_table": osm_table,
            "osm_object": osm_object,
            "custom_poi": custom_poi,
            "alias": alias,
            "type": alias_type
        })

        existing_aliases.add(key)